In [1]:
from importlib import reload
import medvqa

In [20]:
reload(medvqa.models.ensemble.multilabel_ensemble_search)

<module 'medvqa.models.ensemble.multilabel_ensemble_search' from '/home/pamessina/medvqa/medvqa/models/ensemble/multilabel_ensemble_search.py'>

In [2]:
import numpy as np
from medvqa.models.ensemble import MultilabelOptimalEnsembleSearcher

In [3]:
k = 4
n = 10
m = 5
probs = np.random.rand(k, n, m)
gt = np.random.randint(0, 2, (n, m))

In [4]:
probs.shape, gt.shape

((4, 10, 5), (10, 5))

In [5]:
gt

array([[1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 1, 1, 1],
       [1, 1, 0, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 1, 1],
       [1, 0, 1, 0, 1],
       [0, 0, 1, 1, 1]])

In [11]:
mloes = MultilabelOptimalEnsembleSearcher(probs, gt, score_name='prc_auc')

In [13]:
mloes.try_basic_weight_heuristics()

Trying basic weight heuristics...
  1) Try each model alone


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 291.14it/s]


  2) Try pairs of models


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 330.08it/s]

  3) Try the average of all models
  Done


In [14]:
mloes.evaluate_best_predictions()

score=0.5569653250188964


0.5569653250188964

In [15]:
mloes.sample_weights(100)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 422.40it/s]


In [16]:
mloes.sample_weights_from_previous_ones(100)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 331.91it/s]


In [17]:
mloes.evaluate_best_predictions()

score=0.5689990551776265


0.5689990551776265

In [21]:
P = probs.transpose(2, 1, 0)[0]
G = gt.transpose(1, 0)[0]
P, G

(array([[0.14379834, 0.26243938, 0.76658406, 0.79135398],
        [0.20708627, 0.88920296, 0.45012919, 0.03209512],
        [0.67813027, 0.46265779, 0.91703017, 0.34301678],
        [0.20183022, 0.71330381, 0.06806284, 0.22237544],
        [0.41513064, 0.58478495, 0.74702036, 0.37799092],
        [0.74269982, 0.53022088, 0.15216235, 0.49835152],
        [0.38217506, 0.38718614, 0.85134016, 0.98693191],
        [0.51776825, 0.43410479, 0.81180903, 0.66853343],
        [0.05370033, 0.04599556, 0.05297401, 0.33564935],
        [0.10626623, 0.97262455, 0.87907408, 0.35905688]]),
 array([1, 0, 0, 0, 1, 1, 0, 0, 1, 0]))

In [24]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
from scipy.optimize import minimize

# Assume P is your MxN matrix of probabilities
# Assume G is your ground-truth label vector of size N
M = 5
N = 10
P = np.random.rand(M, N)
G = np.random.randint(0, 2, N)

def weighted_average_ensemble(weights, P):
    # weights is a vector of size M
    # P is a matrix of size MxN
    return np.average(P, axis=0, weights=weights)

def prc_auc_score(weights, P, G):
    # Get the weighted average predictions
    ensemble_preds = weighted_average_ensemble(weights, P)
    # Calculate precision-recall curve
    precision, recall, _ = precision_recall_curve(G, ensemble_preds)
    # Calculate AUC
    return -auc(recall, precision)  # Negative because we minimize in optimization

# Initial weights (equal weights)
initial_weights = np.ones(P.shape[0]) / P.shape[0]

# Constraints: weights should sum to 1
constraints = ({'type': 'eq', 'fun': lambda w: 1 - sum(w)})

# Bounds: weights should be between 0 and 1
bounds = [(0, 1) for _ in range(P.shape[0])]

# Optimization
result = minimize(prc_auc_score, initial_weights, args=(P, G), method='SLSQP', bounds=bounds, constraints=constraints)

# Optimal weights
optimal_weights = result.x

# Ensemble predictions using optimal weights
ensemble_predictions = weighted_average_ensemble(optimal_weights, P)

In [29]:
P, ensemble_predictions, optimal_weights

(array([[0.47827167, 0.59811935, 0.86960844, 0.24509691, 0.58580935,
         0.843191  , 0.45777964, 0.97483341, 0.72187809, 0.54398505],
        [0.88490294, 0.49918815, 0.52818815, 0.56698574, 0.49010634,
         0.4057819 , 0.35461876, 0.07430059, 0.31466248, 0.423951  ],
        [0.0851603 , 0.81548073, 0.99937688, 0.25524354, 0.07929188,
         0.09721713, 0.47920024, 0.68515235, 0.85483686, 0.98815449],
        [0.80357453, 0.63788271, 0.57995352, 0.16103289, 0.69228241,
         0.75792728, 0.23938673, 0.70676791, 0.7750239 , 0.57492023],
        [0.77281565, 0.30924471, 0.61881496, 0.96483308, 0.91492042,
         0.72214136, 0.52489136, 0.64917116, 0.06701547, 0.38123213]]),
 array([0.60494502, 0.57198313, 0.71918839, 0.43863843, 0.55248208,
        0.56525173, 0.41117534, 0.61804509, 0.54668336, 0.58244858]),
 array([0.2, 0.2, 0.2, 0.2, 0.2]))

In [31]:
P.shape, G.shape

((5, 10), (10,))

In [41]:
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression()
meta_model.fit(P.T, G)
meta_predictions = meta_model.predict_proba(P.T)[:, 1]

In [43]:
precision, recall, _ = precision_recall_curve(G, meta_predictions)
prc_auc = auc(recall, precision)

In [47]:
meta_model.coef_

array([[ 0.58731864, -0.33928372,  0.83438047,  0.38039845, -0.53600753]])

In [48]:
meta_model.intercept_

array([-0.13903571])

In [44]:
prc_auc

0.9742063492063491